In [2]:
# Import libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import table

# classifiers / models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDClassifier
from sklearn.svm import SVC, LinearSVC

# other
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
import subprocess

In [3]:
#Import data and drop unused index column

df = pd.read_csv("../../data/processed/wrangled.csv")
df.drop(["Unnamed: 0"], axis=1)

age  sex  height  weight  QRS_duration  P-R_interval  Q-T_interval  \
0    75.0  0.0   190.0    80.0          91.0         193.0         371.0   
1    56.0  1.0   165.0    64.0          81.0         174.0         401.0   
2    54.0  0.0   172.0    95.0         138.0         163.0         386.0   
3    55.0  0.0   175.0    94.0         100.0         202.0         380.0   
4    75.0  0.0   190.0    80.0          88.0         181.0         360.0   
5    13.0  0.0   169.0    51.0         100.0         167.0         321.0   
6    40.0  1.0   160.0    52.0          77.0         129.0         377.0   
7    49.0  1.0   162.0    54.0          78.0           0.0         376.0   
8    44.0  0.0   168.0    56.0          84.0         118.0         354.0   
9    50.0  1.0   167.0    67.0          89.0         130.0         383.0   
10   62.0  0.0   170.0    72.0         102.0         135.0         401.0   
11   45.0  1.0   165.0    86.0          77.0         143.0         373.0   
12   54.0  1.0   172.0    58.0          78.0         155.0         382.0   
13   30.0  0.0   170.0    73.0          91.0         180.0         355.0   
14   44.0  1.0   160.0    88.0          77.0         158.0         399.0   
15   47.0  1.0   150.0    48.0          75.0         132.0         350.0   
16   47.0  0.0   171.0    59.0          82.0         145.0         347.0   
17   46.0  1.0   158.0    58.0          70.0         120.0         353.0   
18   73.0  0.0   165.0    63.0          91.0         154.0         392.0   
19   57.0  1.0   166.0    72.0          82.0         181.0         399.0   
20   28.0  1.0   160.0    58.0          83.0         251.0         383.0   
21   45.0  0.0   169.0    67.0          90.0         122.0         336.0   
22   36.0  1.0   153.0    75.0          71.0         132.0         364.0   
23   57.0  1.0   165.0    59.0          75.0         157.0         406.0   
24   40.0  1.0   153.0    55.0          82.0         140.0         388.0   
25   44.0  0.0   169.0    80.0         109.0         128.0         382.0   
26   34.0  0.0   170.0    73.0          94.0         186.0         373.0   
27   31.0  1.0   160.0    54.0          95.0         161.0         407.0   
28   56.0  1.0   164.0    65.0          90.0         164.0         420.0   
29   51.0  1.0   160.0    83.0          96.0         147.0         400.0   
..    ...  ...     ...     ...           ...           ...           ...   
422  29.0  1.0   162.0    57.0          83.0         164.0         359.0   
423  51.0  0.0   186.0    95.0          94.0         203.0         367.0   
424   7.0  0.0   119.0    21.0         140.0         157.0         438.0   
425  36.0  0.0   171.0    93.0          87.0         150.0         362.0   
426  35.0  1.0   160.0    53.0          55.0         163.0         340.0   
427  58.0  0.0   160.0    65.0         133.0         148.0         417.0   
428  64.0  0.0   160.0    63.0          83.0           0.0         364.0   
429   8.0  1.0   130.0    24.0          77.0         125.0         358.0   
430  11.0  0.0   138.0    29.0         123.0         145.0         361.0   
431  47.0  0.0   166.0    56.0          79.0         145.0         381.0   
432  11.0  0.0   140.0    42.0          88.0         123.0         362.0   
433  70.0  0.0   167.0    60.0          80.0         149.0         290.0   
434  20.0  0.0   178.0    65.0          88.0         155.0         360.0   
435  39.0  1.0   164.0    62.0          79.0         155.0         367.0   
436  32.0  1.0   164.0    57.0          77.0         144.0         340.0   
437  35.0  1.0   155.0    63.0          87.0         142.0         391.0   
438  37.0  0.0   175.0    82.0          88.0         146.0         357.0   
439  49.0  1.0   168.0    66.0          94.0         170.0         383.0   
440  37.0  0.0   176.0    72.0          88.0         153.0         389.0   
441  37.0  1.0   160.0    50.0          74.0         143.0         374.0   
442  65.0  1.0   160.0    50.0          85.0         1

In [4]:
# Split data into test, train, and validation sets

feat = df.drop(["class"], axis=1)

X_train_all, X_test, y_train_all, y_test = train_test_split(feat, df["class"], 
                                                    test_size=0.20, random_state=142857)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, 
                                                      test_size=0.20, random_state=142857)

In [5]:
# Drop unused index column

X_train = X_train.drop(["Unnamed: 0"], axis=1)
X_train_all = X_train_all.drop(["Unnamed: 0"], axis=1)
X_valid = X_valid.drop(["Unnamed: 0"], axis=1)
X_test = X_test.drop(["Unnamed: 0"], axis=1)


In [6]:
# Create lists for results comparison

model = ["SVM", "Decision Tree", "kNN"]
v_err = []
t_err = []

In [7]:
# Fit and cross-validate an SVM

svm = SVC(kernel="rbf", random_state=142857)
parameters = {"gamma": 10.0**np.arange(-7,4), "C": 10.0**np.arange(-4,4)}
clf = GridSearchCV(svm, parameters, cv=10)
clf.fit(X_train, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

svm_best = SVC(kernel="rbf", C=clf.best_params_["C"], gamma=clf.best_params_["gamma"], random_state=142857)
svm_best.fit(X_train_all, y_train_all)
svm_best_y = svm_best.predict(X_test)
test_err = 1.0-svm_best.score(X_test, y_test)
print("Test Error:", test_err)

v_err.append(clf.best_score_)
t_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'C': 1000.0, 'gamma': 9.9999999999999995e-08}
Validation Error 0.694444444444
Test Error: 0.340659340659


In [8]:
# Fit and cross-validate a decision tree

dtree = DecisionTreeClassifier(random_state=142857)
parameters = {"max_depth": np.arange(1,10)}
clf = GridSearchCV(dtree, parameters, cv=10)
clf.fit(X_train, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

dtree_best = DecisionTreeClassifier(max_depth=clf.best_params_["max_depth"], random_state=142857)
dtree_best.fit(X_train_all, y_train_all)
dtree_best_y = dtree_best.predict(X_test)
test_err = 1.0-dtree_best.score(X_test, y_test)
print("Test Error:", test_err)

v_err.append(clf.best_score_)
t_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'max_depth': 5}
Validation Error 0.694444444444
Test Error: 0.384615384615


In [9]:
# Fit and cross-validate k-nearest neighbors

knn = KNeighborsClassifier()
parameters = {"n_neighbors": np.arange(1,10)}
clf = GridSearchCV(knn, parameters, cv=10)
clf.fit(X_train, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

knn_best = KNeighborsClassifier(n_neighbors=clf.best_params_["n_neighbors"])
knn_best.fit(X_train_all, y_train_all)
knn_best_y = knn_best.predict(X_test)
test_err = 1.0-knn_best.score(X_test, y_test)
print("Test Error:", test_err)

v_err.append(clf.best_score_)
t_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'n_neighbors': 3}
Validation Error 0.618055555556
Test Error: 0.428571428571


In [10]:
# Select features using RFE on three different types of regressions

rfe = RFE(estimator = LinearRegression())
rfe.fit(X_train, y_train)
X_train
X_train_lm = X_train.ix[:, rfe.support_]
X_valid_lm = X_valid.ix[:, rfe.support_]

rfe = RFE(estimator = Lasso())
rfe.fit(X_train, y_train)
X_train
X_train_L1 = X_train.ix[:, rfe.support_]
X_valid_L1 = X_valid.ix[:, rfe.support_]

rfe = RFE(estimator = Ridge())
rfe.fit(X_train, y_train)
X_train
X_train_L2 = X_train.ix[:, rfe.support_]
X_valid_L2 = X_valid.ix[:, rfe.support_]

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [11]:
# Create lists for results comparison

selection = ["Linear", "L1", "L2"]
v_svm_err = []
t_svm_err = []

v_dtree_err = []
t_dtree_err = []

In [12]:
# Fit and cross-validate an SVM with RFE features linearly selected

svm = SVC(kernel="rbf", random_state=142857)
parameters = {"gamma": 10.0**np.arange(-7,4), "C": 10.0**np.arange(-4,4)}
clf = GridSearchCV(svm, parameters, cv=10)
clf.fit(X_train_lm, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

svm_best = SVC(kernel="rbf", C=clf.best_params_["C"], gamma=clf.best_params_["gamma"], random_state=142857)
svm_best.fit(X_train_all, y_train_all)
svm_best_y = svm_best.predict(X_test)
test_err = 1.0-svm_best.score(X_test, y_test)
print("Test Error:", test_err)

v_svm_err.append(clf.best_score_)
t_svm_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'C': 1000.0, 'gamma': 9.9999999999999995e-07}
Validation Error 0.684027777778
Test Error: 0.362637362637


In [13]:
# Fit and cross-validate an SVM with RFE features L1 regularized

svm = SVC(kernel="rbf", random_state=142857)
parameters = {"gamma": 10.0**np.arange(-7,4), "C": 10.0**np.arange(-4,4)}
clf = GridSearchCV(svm, parameters, cv=10)
clf.fit(X_train_L1, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

svm_best = SVC(kernel="rbf", C=clf.best_params_["C"], gamma=clf.best_params_["gamma"], random_state=142857)
svm_best.fit(X_train_all, y_train_all)
svm_best_y = svm_best.predict(X_test)
test_err = 1.0-svm_best.score(X_test, y_test)
print("Test Error:", test_err)

v_svm_err.append(clf.best_score_)
t_svm_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'C': 10.0, 'gamma': 1.0000000000000001e-05}
Validation Error 0.694444444444
Test Error: 0.351648351648


In [14]:
# Fit and cross-validate an SVM with RFE features L2 regularized

svm = SVC(kernel="rbf", random_state=142857)
parameters = {"gamma": 10.0**np.arange(-7,4), "C": 10.0**np.arange(-4,4)}
clf = GridSearchCV(svm, parameters, cv=10)
clf.fit(X_train_L2, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

svm_best = SVC(kernel="rbf", C=clf.best_params_["C"], gamma=clf.best_params_["gamma"], random_state=142857)
svm_best.fit(X_train_all, y_train_all)
svm_best_y = svm_best.predict(X_test)
test_err = 1.0-svm_best.score(X_test, y_test)
print("Test Error:", test_err)

v_svm_err.append(clf.best_score_)
t_svm_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'C': 100.0, 'gamma': 9.9999999999999995e-07}
Validation Error 0.6875
Test Error: 0.351648351648


In [15]:
# Fit and cross-validate a decision tree with RFE features linearly selected

dtree = DecisionTreeClassifier(random_state=142857)
parameters = {"max_depth": np.arange(1,10)}
clf = GridSearchCV(dtree, parameters, cv=10)
clf.fit(X_train_lm, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

dtree_best = DecisionTreeClassifier(max_depth=clf.best_params_["max_depth"], random_state=142857)
dtree_best.fit(X_train_all, y_train_all)
dtree_best_y = dtree_best.predict(X_test)
test_err = 1.0-dtree_best.score(X_test, y_test)
print("Test Error:", test_err)

v_dtree_err.append(clf.best_score_)
t_dtree_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'max_depth': 4}
Validation Error 0.631944444444
Test Error: 0.384615384615


In [16]:
# Fit and cross-validate a decision tree with RFE features L1 regularized

dtree = DecisionTreeClassifier(random_state=142857)
parameters = {"max_depth": np.arange(1,10)}
clf = GridSearchCV(dtree, parameters, cv=10)
clf.fit(X_train_L1, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

dtree_best = DecisionTreeClassifier(max_depth=clf.best_params_["max_depth"], random_state=142857)
dtree_best.fit(X_train_all, y_train_all)
dtree_best_y = dtree_best.predict(X_test)
test_err = 1.0-dtree_best.score(X_test, y_test)
print("Test Error:", test_err)

v_dtree_err.append(clf.best_score_)
t_dtree_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'max_depth': 4}
Validation Error 0.673611111111
Test Error: 0.384615384615


In [17]:
# Fit and cross-validate a decision tree with RFE features L2 regularized

dtree = DecisionTreeClassifier(random_state=142857)
parameters = {"max_depth": np.arange(1,10)}
clf = GridSearchCV(dtree, parameters, cv=10)
clf.fit(X_train_L2, y_train)

print(clf.best_params_)
print("Validation Error", clf.best_score_)

dtree_best = DecisionTreeClassifier(max_depth=clf.best_params_["max_depth"], random_state=142857)
dtree_best.fit(X_train_all, y_train_all)
dtree_best_y = dtree_best.predict(X_test)
test_err = 1.0-dtree_best.score(X_test, y_test)
print("Test Error:", test_err)

v_dtree_err.append(clf.best_score_)
t_dtree_err.append(test_err)

/home/indiana-nikel/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'max_depth': 4}
Validation Error 0.638888888889
Test Error: 0.384615384615


In [18]:
# Create datatables

df_init = pd.DataFrame({"Model": model, "Validation Error": v_err, "Training Error": t_err})
df_svm = pd.DataFrame({"Selection Process": selection, "Validation Error": v_svm_err, "Training Error": t_svm_err})
df_dtree = pd.DataFrame({"Selection Process": selection, "Validation Error": v_dtree_err, "Training Error": t_dtree_err})

In [21]:
# Export datatables, pulled from https://stackoverflow.com/questions/35634238/how-to-save-a-pandas-dataframe-table-as-a-png

df_init.to_html('init.html')
subprocess.call(
    'wkhtmltoimage -f png --width 0 init.html init.png', shell=True)

df_init.to_html('svm.html')
subprocess.call(
    'wkhtmltoimage -f png --width 0 svm.html svm.png', shell=True)

df_init.to_html('dtree.html')
subprocess.call(
    'wkhtmltoimage -f png --width 0 dtree.html dtree.png', shell=True)

0